In [1]:
import os
import re
import utils

import numpy as np
import tensorflow as tf

from IPython.display import clear_output

In [2]:
from tensorflow.keras.models import load_model 
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config = config)

In [3]:
# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [4]:
data_dir = 'moes_tavern_lines.txt'
text = utils.load_data(data_dir)

# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [5]:
view_sentence_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

Dataset Stats
Roughly the number of unique words: 11492


In [6]:
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))

Number of scenes: 262


In [7]:
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

Average number of sentences in each scene: 15.248091603053435


In [8]:
sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))

Number of lines: 4257


In [9]:
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

Average number of words in each line: 11.50434578341555


In [10]:
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




In [11]:
text.replace('&','and')
clear_output()

In [12]:
text = re.sub(r'[^\w\s]','',text)
clear_output()

In [13]:
text = ''.join([i.lower() for i in text if not i.isdigit()])
clear_output()

In [14]:
# The unique characters in the file

vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

36 unique characters


In [15]:
# Vectorize the text

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [16]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '_' :   2,
  'a' :   3,
  'b' :   4,
  'c' :   5,
  'd' :   6,
  'e' :   7,
  'f' :   8,
  'g' :   9,
  'h' :  10,
  'i' :  11,
  'j' :  12,
  'k' :  13,
  'l' :  14,
  'm' :  15,
  'n' :  16,
  'o' :  17,
  'p' :  18,
  'q' :  19,
  ...
}


In [17]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'moe_szyslak i' ---- characters mapped to int ---- > [15 17  7  2 21 28 27 21 14  3 13  1 11]


In [18]:
# Build Training Data

In [19]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

m
o
e
_
s


In [20]:
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'moe_szyslak into phone moes tavern where the elite meet to drink\nbart_simpson eh yeah hello is mike t'
'here last name rotch\nmoe_szyslak into phone hold on ill check to barflies mike rotch mike rotch hey h'
'as anybody seen mike rotch lately\nmoe_szyslak into phone listen you little puke one of these days im '
'gonna catch you and im gonna carve my name on your back with an ice pick\nmoe_szyslak whats the matter'
' homer youre not your normal effervescent self\nhomer_simpson i got my problems moe give me another on'


#### For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [21]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [22]:
# Print the first examples input and target values:

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'moe_szyslak into phone moes tavern where the elite meet to drink\nbart_simpson eh yeah hello is mike '
Target data: 'oe_szyslak into phone moes tavern where the elite meet to drink\nbart_simpson eh yeah hello is mike t'


In [23]:
# For one charachter predict next one, then concatenate context through time

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 15 ('m')
  expected output: 17 ('o')
Step    1
  input: 17 ('o')
  expected output: 7 ('e')
Step    2
  input: 7 ('e')
  expected output: 2 ('_')
Step    3
  input: 2 ('_')
  expected output: 21 ('s')
Step    4
  input: 21 ('s')
  expected output: 28 ('z')


In [24]:
# Batch size
BATCH_SIZE = 32

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

dataset

<BatchDataset shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

In [25]:
# Params

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape = [batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences = True,
                        stateful = True,
                        recurrent_initializer = 'glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [27]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim = embedding_dim,
  rnn_units = rnn_units,
  batch_size = BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 256)           9216      
_________________________________________________________________
lstm (LSTM)                  (32, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (32, None, 36)            36900     
Total params: 5,293,092
Trainable params: 5,293,092
Non-trainable params: 0
_________________________________________________________________


In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(32, 100, 36) # (batch_size, sequence_length, vocab_size)


In [29]:
# Check for one example

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples = 1)
sampled_indices = tf.squeeze(sampled_indices, axis = -1).numpy()

In [30]:
sampled_indices

array([ 0, 18,  4,  6, 19,  2, 14, 15, 32, 22, 33, 20, 19, 26, 29,  0,  1,
       12, 18, 22, 28,  5, 34, 23,  7, 20, 17, 24,  4, 23,  9,  6, 24, 27,
       17, 14, 31,  6, 25,  4, 30,  8, 29,  1, 31,  3, 19, 32, 25, 14, 10,
       14, 30, 17,  0,  4, 11,  6,  0, 24,  3, 13, 15, 10, 24,  6, 12, 29,
       14,  9,  6,  7, 27, 21, 26,  7, 17, 34,  0, 13, 33,  7, 32,  8, 28,
        7, 28, 13, 12,  1, 12,  8,  2, 10, 29, 18,  4,  2, 35, 34])

In [31]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0].numpy()])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 're of speech\nmoe_szyslak hiya homer sighs\nhomer_simpson whats the matter moe\nmoe_szyslak ah business'

Next Char Predictions: 
 '\npbdq_lmètérqxà\n jptzcóuerovbugdvyolädwbãfà äaqèwlhlão\nbid\nvakmhvdjàlgdeysxeoó\nkéeèfzezkj jf_hàpb_üó'


In [32]:
# Train

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (32, 100, 36)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.5827286


In [33]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.001), loss = loss)

In [34]:
# Callbacks

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

In [35]:
EPOCHS = 25

In [36]:
history = model.fit(dataset, epochs = EPOCHS, callbacks = [checkpoint_callback])

Train for 87 steps
Epoch 1/25
87/87 [==============================] - 6s 67ms/step - loss: 2.7512
Epoch 2/25
87/87 [==============================] - 6s 65ms/step - loss: 2.0717
Epoch 3/25
87/87 [==============================] - 5s 54ms/step - loss: 1.8318
Epoch 4/25
87/87 [==============================] - 5s 57ms/step - loss: 1.6751
Epoch 5/25
87/87 [==============================] - 5s 54ms/step - loss: 1.5538
Epoch 6/25
87/87 [==============================] - 5s 53ms/step - loss: 1.4642
Epoch 7/25
87/87 [==============================] - 5s 53ms/step - loss: 1.3882
Epoch 8/25
87/87 [==============================] - 5s 56ms/step - loss: 1.3244
Epoch 9/25
87/87 [==============================] - 5s 55ms/step - loss: 1.2645
Epoch 10/25
87/87 [==============================] - 5s 60ms/step - loss: 1.2065
Epoch 11/25
87/87 [==============================] - 5s 57ms/step - loss: 1.1484
Epoch 12/25
87/87 [==============================] - 5s 57ms/step - loss: 1.0866
Epoch 13/25
87/87 

#### To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [37]:
# Load Checkpoint

tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_25'

In [38]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            9216      
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 36)             36900     
Total params: 5,293,092
Trainable params: 5,293,092
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Get Pred Func

def generate_text(model, start_string):
  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.5

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [41]:
print(generate_text(model, start_string = u"homer_simpson "))

homer_simpson not you but i wont get this the word friends we met through dr hibbert at a party at beer please
homer_simpson you still world chief wiggum
kent_brockman another world of extract first head suddenly interesting moe id say thats a pretty strong endorsement so how about you and me
moe_szyslak oh yeah here ya bobither like they love this suppose is d he invented kidding i know this will read a list of senatorswith possibly more senatorsto be named later by other senators
kent_brockman desperate what are you youre like a single you dont have a you homer care about the corner and head down
homer_simpson hey what happened to the barney you gotta fly us up there and save my kids
barney_gumble sings ive been working on the railroad  all the live long day then yeah thats what i get my cash uh look i just wanna tell you all that scenee
barney_gumble we want chilly willy we want chilly willy
bart_simpson hmmm the only problem is it to moe loud seymour
homer_simpson perplexed moe wev